# Test box.py library

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

%matplotlib inline
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram

import m2lib22.sat as sat

import warnings
warnings.filterwarnings("ignore")

In [3]:
if False:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster(processes=4, cores=4)
    w = cluster.scale(jobs=5)
else:
    from dask.distributed import Client, LocalCluster
    #
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 56,Total memory: 100.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38539,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 56
Started: Just now,Total memory: 100.00 GiB
Comm: tcp://127.0.0.1:46117,Total threads: 7
Dashboard: http://127.0.0.1:35649/status,Memory: 12.50 GiB
Nanny: tcp://127.0.0.1:56787,


---
## load data

Need to chunk a bit along `obs` dimension

- `site_obs`: drifter time
- `sassa_time`: satellite time


In [4]:
# A deplacer dans la bibliothèque

import os
from glob import glob

colloc_path = {2016: '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/mdb/2016/',
               2018: '/home/datawork-cersat-public/cache/users/jfpiolle/felyx/drifters/mdb/2018/',
              }

def load_collocalisations(source, satellite=None, drifter=None):
    """ Load collocalisations altimetry / GDP data
    
    Parameters
    ----------
    source : str, int
        Data source, e.g. 2016, 2018
    satellite : str
        type of satellite we want to select among "SARAL" or "Sentinel"  (if we want to) 
    drifter : str
        type of drifter we want to select among "gps" or "argos" (if we want to) 
    
    Return
    ------
    list of .nc files
    
    """
    files = sorted(glob(os.path.join(colloc_path[source], "*.nc")))
    
    if satellite is not None:
        files = [f for f in files if satellite in f]
    if drifter is not None:
        files = [f for f in files if drifter in f]
        
    return files

In [7]:
def change_prefix(ds):
    """ Find the data prefixes
    
    Parameters
    ----------
    ds : xr.DataArray
        colocalisations dataset
        
    Return
    ------
    drifters prefix, satellite prefix : str, str
    
    """
    argos = any(["argos" in variable for variable in list(ds)])
    #gps = any(["gps" in variable for variable in list(ds)])

    if argos:
        ds = ds.rename({v: v.replace("argos_drifters", "drifter") for v in ds})
    else:
        ds = ds.rename({v: v.replace("gps_drifters", "drifter") for v in ds})
    
    ds = ds.rename({v: v.replace("sassa", "alti") for v in ds})
    ds = ds.rename_dims({"sassa_time" :"alti_time"})
    return ds

In [8]:
#load_collocalisations(2018)
nc_files = load_collocalisations(2018, drifter='gps')
nc = nc_files[0]
ds = (xr.open_dataset(nc)
      .chunk(dict(obs=1000))
      #.persist()
      
     )
ds=change_prefix(ds)

# add several variables in coords
ds = (ds
      .set_coords(["drifter_"+d for d in ["time", "lon", "lat"]])
      .set_coords(["alti_"+d for d in ["time_", "lon", "lat"]])
     )


print('Dataset size = %.1f GB' %(ds.nbytes/1e9))
#nc_files


Dataset size = 1.3 GB


In [9]:
ds

<xarray.Dataset>
Dimensions:                        (obs: 5520, site_obs: 1489, alti_time: 21)
Coordinates:
    time                           (obs) datetime64[ns] dask.array<chunksize=(1000,), meta=np.ndarray>
    lon                            (obs) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    lat                            (obs) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    drifter_time                   (obs, site_obs) datetime64[ns] dask.array<chunksize=(1000, 1489), meta=np.ndarray>
    drifter_lon                    (obs, site_obs) float64 dask.array<chunksize=(1000, 1489), meta=np.ndarray>
    drifter_lat                    (obs, site_obs) float64 dask.array<chunksize=(1000, 1489), meta=np.ndarray>
    alti_lat                       (obs, alti_time) float64 dask.array<chunksize=(1000, 21), meta=np.ndarray>
    alti_time_                     (obs, alti_time) datetime64[ns] dask.array<chunksize=(1000, 21), meta=np.ndarray>
    alti_lon                       (obs, alti_time) float64 dask.array<chunksize=(1000, 21), meta=np.ndarray>
Dimensions without coordinates: obs, site_obs, alti_time
Data variables: (12/43)
    __site_id                      (obs) object dask.array<chunksize=(1000,), meta=np.ndarray>
    __site_name                    (obs) object dask.array<chunksize=(1000,), meta=np.ndarray>
    drifter_rowsize                (obs, site_obs) float64 dask.array<chunksize=(1000, 1489), meta=np.ndarray>
    drifter_WMO                    (obs, site_obs) float64 dask.array<chunksize=(1000, 1489), meta=np.ndarray>
    drifter_expno                  (obs, site_obs) float64 dask.array<chunksize=(1000, 1489), meta=np.ndarray>
    drifter_deploy_date            (obs, site_obs) datetime64[ns] dask.array<chunksize=(1000, 1489), meta=np.ndarray>
    ...                             ...
    alti_sla                       (obs, alti_time) float64 dask.array<chunksize=(1000, 21), meta=np.ndarray>
    alti___distance                (obs) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    alti_swh                       (obs, alti_time) float64 dask.array<chunksize=(1000, 21), meta=np.ndarray>
    alti_sla_denoised              (obs, alti_time) float64 dask.array<chunksize=(1000, 21), meta=np.ndarray>
    alti_miniprod_content_mask     (obs, alti_time) float32 dask.array<chunksize=(1000, 21), meta=np.ndarray>
    alti___source_center_index     (obs) object dask.array<chunksize=(1000,), meta=np.ndarray>
Attributes:
    __id:                   SASSA-L3-SLA-SARAL__gps
    __time_coverage_end:    2018-02-01T00:00:00
    __time_coverage_start:  2018-01-01T00:00:00

### add local coordinates oriented along the altimetric track

In [ ]:
xr.apply_ufunc?

In [ ]:
def get_proj(lonc, latc):#converts from lon, lat to native map projection x,y
    """ Create pyproj Proj object, project is an azimutal Eqsuidistant projection centered on the central point of the selected satellite track = matching point
    https://proj.org/operations/projections/aeqd.html
            
    Parameters
    ----------
    lonc,latc : float
        central longitude and latitude of the satellite track, matching point on which the box will be centered
    Return
    ------
    pyproj.Proj object
    """
    return pyproj.Proj(proj="aeqd", lat_0=latc, lon_0=lonc, datum="WGS84", units="m")#aeqd Azimutal EQuiDistant projection centered on lonc,latc

def compute_box_orientation(lonc, latc, lon1, lat1):
    """Compute the orientation of the box i.e. the angle (-180 and 180°) between the oriented track (time=0 -> end) and the longitude axe.
        
    Parameters
    ----------
    lonc,latc : float
        central longitude and latitude of the satellite track, matching point on which the box will be centered
    lon1,lat1 : float
        longitude and latitude of the end of the satellite track
    Return
    ------
    box orientation
    """
    proj = get_proj(lonc, latc)
    # get local coordinate
    xc, yc = proj.transform(lonc, latc) #xc=yc=0 origin of the box grid
    x1, y1 = proj.transform(lon1, lat1)
    # get orientation of defined by central point and point 1
    #print(lonc,latc,xc,yc,x1,y1)
    phi = np.arctan2( y1-yc, x1-xc )*180/np.pi # angle (-180 and 180°) between the oriented track (time=0 -> end) and the longitude axe
    return phi
    
#compute_box_vec = np.vectorize(compute_box_core)

def compute_box(ds):
    """ Compute box properties around each collocation: 
    central position and orientation
    
    Parameters
    ----------
    ds : xr.DataArray
        colocalisations dataset
    asuff : str
        satellite prefixe

    Return
    ------
    central longitude, central latitude, orientation : xr.Datarray, xr.Datarray, xr.Datarray
    """
    i = (ds["alti_time"].size-1)//2
    lonc = ds["alti_lon"].isel(alti_time=i)
    latc = ds["alti_lat"].isel(alti_time=i)
    lon1 = ds["alti_lon"].isel(alti_time=-1)
    lat1 = ds["alti_lat"].isel(alti_time=-1)
    end_pt = True
    
    if (np.isnan(lon1) or np.isnan(lat1)):
        lon1 = ds_pb_one['alti_lon'].isel(alti_time=0).compute()
        lat1 = ds_pb_one['alti_lat'].isel(alti_time=0).compute()
        end_pt = False

    # will need to vectorize with dask eventually
    phi = xr.apply_ufunc(compute_box_orientation,
                         lonc, latc,
                         lon1, lat1,
                         #input_core_dims = [[]]*4,
                         vectorize=True,
                         dask="parallelized",
                        ).rename("phi")
    if not end_pt : 
        phi = 180+phi
    #phi = compute_box_vec(lonc, latc, lon1, lat1)    

    return lonc, latc, phi

In [ ]:
def compute_box_grid_lonlat_core(lonc, latc, phi, x=None, y=None):
    """ Compute coordinates longitude, latitude of the local box grid (x-along satellite track, y-normal to satellite track -> lon,lat)
    https://github.com/rasterio/affine
    
    Parameters
    ----------
    lonc, latc, phi: float  central position and orientation of the box
    x, y: np.array local grid of the box (with origin at (lonc, latc) and x-axis aligned 
    with the satellite track (lonc, latc) - (lon1, lat1) direction)
    
    Return
    ------
    longitude, latitude : np.array, np.array
    
    """
    xv, yv = np.meshgrid(x, y)
    proj = get_proj(lonc, latc)
    #assert False, help(proj.transform)
    xc, yc = proj.transform(lonc, latc)
    # apply inverse affine transformation
    a_fwrd = Affine.translation(-xc, -yc) * Affine.rotation(-phi, pivot=(xc, yc))
    a_back = ~a_fwrd
    #compute coordinates of x,y in the lon, lat orientated grid
    xv_inv, yv_inv = a_back * (xv, yv) 
    lon, lat = proj.transform(xv_inv, yv_inv, 
                              direction=pyproj.enums.TransformDirection.INVERSE,
                             )
    return lon, lat

def compute_box_grid_lonlat(ds, x, y):
    """ Compute local coordinates in longitude, latitude of the local box grid (x-along satellite track, y-normal to satellite track -> lon,lat) for all colocalisations
    
     Parameters
    ----------
    ds : xr.DataArray
        colocalisations dataset 
    x, y: np.array      
        local grid of the box (with origin at (lonc, latc) and x-axis aligned )
    
    Return
    ------
    longitude, latitude : xr.Datarray, xr.Datarray
      
    """
    lon, lat = xr.apply_ufunc(compute_box_grid_lonlat_core,
                   ds.box_lonc, ds.box_latc, ds.box_phi,
                   kwargs=dict(x=x, y=y),
                   output_core_dims = [["box_y", "box_x"]]*2,
                   vectorize=True,
                              dask="parallelized",
                              dask_gufunc_kwargs=dict(output_sizes=dict(box_x=x.size, box_y=y.size)),
                  )
    lon = lon.assign_coords(box_x=x, box_y=y)
    lat = lat.assign_coords(box_x=x, box_y=y)
    return lon, lat

In [ ]:
def lonlat2xy(lonc, latc, phi, lon, lat, lon1=None, lat1=None):
    """ return coordinates with origin at (lonc, latc) and x-axis aligned 
    with (lonc, latc) - (lon1, lat1) direction (lon,lat -> x-along satellite track, y-normal to satellite track)
    
    Parameters
    ----------
    lonc, latc, phi: float  
        central position and orientation of the box
    lon, lat : np.array, np.array     
        local grid of the box
    lon1,lat1 : float      
        end of the satellite track
    
    Return
    ------
    local x, local y : np.array, np.array
      
    """
    proj = get_proj(lonc, latc)
    # get local coordinate
    xc, yc = proj.transform(lonc, latc)    
    xl, yl = proj.transform(lon, lat)
    if phi is None:
        # compute phi from x1 and y1
        x1, y1 = proj.transform(lon1, lat1)
        # get orientation of defined by central point and point 1
        phi = np.arctan2( y1-yc, x1-xc )*180/np.pi
    # build affine operators
    a_fwrd = Affine.translation(-xc, -yc) * Affine.rotation(-phi, pivot=(xc, yc))
    #a_back = ~a_fwrd
    
    x, y = a_fwrd * (xl, yl)
    
    return x, y

def compute_local_xy(ds,prefixe, tdim):
    """return coordinates with origin at (lonc, latc) and x-axis aligned
    with (lonc, latc) - (lon1, lat1) direction (lon,lat -> x-along satellite track, y-normal to satellite track) for all colocalisations
    
    Parameters
    ----------
    ds: xr.DataArray
        colocalisations dataset
    datatype: 'drifter' or 'alti'
    
    Return
    ------
    local x, local y : xr.Datarray, xr.Datarray
      
    """

    lon = ds[prefixe+"_lon"]
    lat = ds[prefixe+"_lat"]

    # will need to vectorize with dask eventually
    x, y = xr.apply_ufunc(lonlat2xy,
                   ds.box_lonc, ds.box_latc, ds.box_phi,
                   lon, lat,
                   input_core_dims = [[]]*3+[[tdim]]*2,
                   output_core_dims = [[tdim]]*2,
                   vectorize=True,
                          dask="parallelized",
                  )
    return x, y

In [ ]:
def compute_box_thetas(ds):
        """ Compute the local angles theta between e_lon and x and between e_lat and x for the box of one colocalisation
    https://github.com/rasterio/affine
    
    Parameters
    ----------
    ds : xr.DataArray
        colocalisations dataset 
    Return
    ------
    box_theta_longitude, box_theta_latitude : np.array, np.array
    
    
    """
    dlon_dx, dlon_dy = ds.box_lon.differentiate("box_x"), ds.box_lon.differentiate("box_y")
    dlat_dx, dlat_dy = ds.box_lat.differentiate("box_x"), ds.box_lat.differentiate("box_y")
    return np.arctan2(-dlat_dx, dlat_dy),np.arctan2( dlon_dx,-dlon_dy)

In [ ]:
def compute_drifters_thetas_core(lonc, latc, phi,lonxy, latxy, x, y, dx=100, dy=100):
    """ Compute the local angles theta between e_lon and x and between e_lat and x for drifters locations of one colocalisation
    https://github.com/rasterio/affine
    
    Parameters
    ----------
    lonc, latc, phi: float  
    central position and orientation of the box
    lonxy, latxy : drifters position in longitude, latitude
    x, y: np.array position of drifters in local coordinates (with origin at (lonc, latc) and x-axis aligned 
    with the satellite track (lonc, latc) - (lon1, lat1) direction)
    dx, dy : differential, default is 100m
    
    Return
    ------
    theta_longitude, theta_latitude : np.array, np.array
    
    
    """
    proj = get_proj(lonc, latc)
    #assert False, help(proj.transform)
    xc, yc = proj.transform(lonc, latc)
    # apply inverse affine transformation
    a_fwrd = Affine.translation(-xc, -yc) * Affine.rotation(-phi, pivot=(xc, yc))
    a_back = ~a_fwrd
    #compute coordinates of x,y in the lon, lat orientated grid
    x_invx, y_invx = a_back * (x+dx, y) 
    lonx, latx = proj.transform(x_invx, y_invx, direction=pyproj.enums.TransformDirection.INVERSE,)
    dlon_dx, dlat_dx=(lonx-lonxy)/dx,(latx-latxy)/dx
    
    x_invy, y_invy = a_back * (x, y+dy) 
    lony, laty = proj.transform(x_invy, y_invy, direction=pyproj.enums.TransformDirection.INVERSE,)
    dlon_dy, dlat_dy= (lony-lonxy)/dy,(laty-latxy)/dy
    
    theta_lon = np.arctan2(-dlat_dx, dlat_dy) # angles between e_lon and x
    theta_lat = np.arctan2( dlon_dx,-dlon_dy) # angles between e_lat and x
    return theta_lon, theta_lat


def compute_drifters_thetas(ds,dx=100,dy=100):
    """ Compute the local angles theta between e_lon and x and between e_lat and x for drifters locations for all colocalisations
    
     Parameters
    ----------
    ds : xr.DataArray
        colocalisations dataset 
    dx, dy : differential, default is 100m
        
    Return
    ------
    theta_lon, theta_latitude : xr.Datarray, xr.Datarray
      
    """
    tdim='site_obs'
    theta_lon, theta_lat = xr.apply_ufunc(compute_drifters_thetas_core,
                   ds.box_lonc, ds.box_latc, ds.box_phi,ds.drifter_lon,ds.drifter_lat,ds.drifter_x,ds.drifter_y,dx,dy,
                                          input_core_dims = [[]]*3+[[tdim]]*4+[[]]*2,
                                          output_core_dims = [[tdim]]*2,
                                          vectorize=True,
                                          dask="parallelized",
                                         )
    return theta_lon, theta_lat

In [ ]:
def vevn2vxvy(theta_lon, theta_lat, ve, vn):
    """ Compute velocities projected on the local box grid (ve, vn -> vx-along satellite track, vy-normal to satellite track)
    https://github.com/rasterio/affine
    
    Parameters
    ----------
    lonc, latc, phi: float  central position and orientation of the box
    ve,vn: np.array velocities on lon, lat grid
    theta_lon, theta lat
        
    Return
    ------
    local vx, local vy : np.array, np.array
      
    """
    return ve*np.cos(theta_lon)+vn*np.cos(theta_lat),ve*np.sin(theta_lon)+vn*np.sin(theta_lat)


def compute_local_drifters_velocities(ds,tdim):
    """ Compute local coordinates lon, lat of the local grid of the box (x-along satellite track, y-normal to satellite track -> lon,lat) for all colocalisations
    
     Parameters
    ----------
    ds : xr.DataArray  
        colocalisations dataset
    suff : str        
        data prefix
    tdim : str
        time dimension of data with prefix suff
    
    Return
    ------
    local vx, local vy : xr.Datarray, xr.Datarray
      
    """
    ve=ds["drifter_ve"]
    vn=ds["drifter_vn"]
    
    # will need to vectorize with dask eventually
    vx, vy = xr.apply_ufunc(vevn2vxvy,
                          ds.drifter_theta_lon, ds.drifter_theta_lat,
                          ve, vn,
                          input_core_dims = [[tdim]]*2+[[tdim]]*2,
                          output_core_dims = [[tdim]]*2,
                          vectorize=True,
                            dask="parallelized",
                         )
    return vx, vy

In [ ]:
# define the local grid
x = np.arange(-200e3,200e3,5e3)
y = np.arange(-100e3,100e3,5e3)
ds = ds.persist()


ds["box_lonc"], ds["box_latc"], ds["box_phi"] = compute_box(ds)
ds["box_lon"], ds["box_lat"] = compute_box_grid_lonlat(ds, x, y)
ds["drifter_x"], ds["drifter_y"] = compute_local_xy(ds,'drifter', "site_obs")
ds = ds.set_coords(["drifter_x", "drifter_y"])

ds["alti_x"], ds["alti_y"] = compute_local_xy(ds,'alti',"alti_time")
ds = ds.set_coords(["alti_x", "alti_y"])

ds["box_theta_lon"],ds["box_theta_lat"] = compute_box_thetas(ds) # angles between e_lon and x, angles between e_lat and x
ds['drifter_theta_lon'],ds['drifter_theta_lat']=compute_thetas_drifters(ds)

ds["drifter_vx"], ds["drifter_vy"] = compute_local_drifters_velocities(ds, "site_obs")

# rotate driter velocities: create new variables vx, vy
# def rotate(u,v,phi):
#   # d!! deg/rad
#   return u*np.cos(phi) +- v... , ...

### add along-track sla gradient

In [ ]:
t = ds["alti_time"]
tc = (t + t.shift({"alti_time": -1}))*0.5 # C
tc = tc[:-1].rename({"alti_time": "alti_time_mid"})

ds = ds.assign_coords(**{"alti_time": t, "alti_time_mid": tc})
grid = Grid(ds, coords={"t": {"center": "alti_time_mid", "outer": "alti_time"}})

ds["alti_x_mid"] = grid.interp(ds["alti_x"], axis="t")
ds["alti_y_mid"] = grid.interp(ds["alti_y"], axis="t")
ds = ds.set_coords(["alti_x_mid","alti_y_mid"])

In [ ]:
#dt = grid.diff(ds[asuff+"_time_"], axis="t")/pd.Timedelta("1S")
dx = grid.diff(ds["alti_x"], axis="t")
dy = grid.diff(ds["alti_y"], axis="t")
dxy = np.sqrt(dx**2+dy**2)

g = 9.81
dsla = grid.diff(ds["alti_sla"], axis="t")
ds["g_grad_sla"] = g*dsla/dxy

dsla = grid.diff(ds["alti_sla_denoised"], axis="t")
ds["g_grad_sla_denoised"] = g*dsla/dxy

# add Coriolis frequency
ds["f"] = 2 *2*np.pi/86164.1 * np.sin(ds.lat*np.pi/180)

In [ ]:
ds

### add momentum conservation equation term

In [ ]:
dt=3600
ds['acc_x'] = ds.drifter_vx.differentiate("site_obs")/dt
ds['acc_y'] = ds.drifter_vy.differentiate("site_obs")/dt
ds['coriolis_x'] = -ds["drifter_vy"]*ds.f
ds['coriolis_y'] = ds["drifter_vx"]*ds.f